In [1]:
%config IPCompleter.greedy=True

In [51]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import matplotlib as plt
from sklearn.datasets import make_blobs
#from tensorflow import keras

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [32]:
# What variable(s) are considered the target for your model?
y_var = charity_df.IS_SUCCESSFUL
y_var

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [5]:
# What variable(s) are considered to be the features and what variable are neither for your model?
X = charity_df.drop(columns=['IS_SUCCESSFUL', 'EIN', 'NAME'])
X

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000


In [27]:
# Using the methods described in this module, preprocess all numerical and categorical variables, as needed:
string_dtypes = X.dtypes[X.dtypes == "object"].index.tolist()
X[string_dtypes].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION             4
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [7]:
class_count = X.CLASSIFICATION.value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2500        1
C2190        1
C1732        1
C1900        1
C1370        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# Determine which values to replace
replace_class = list(class_count[class_count < 4000].index)

# Replace in DataFrame
for classification in replace_class:
    X.CLASSIFICATION = X.CLASSIFICATION.replace(classification,"Other")


# Check to make sure binning was successful
X.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [28]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(X[string_dtypes]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(string_dtypes)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [31]:
# Merge the two DataFrames together
charity_relevent_df = pd.merge(X, encode_df, left_index=True, right_index=True).drop(string_dtypes, 1)
charity_relevent_df.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [34]:
# Split our preprocessed data into our features and target arrays
y = y_var
X = charity_relevent_df

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [35]:
# Standardize numerical variables using TensorFlow’s StandardScaler class
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [52]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.467


In [67]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train.values, epochs=100,#callbacks=[cp_callback]
                  )
# Check the structure of the model
nn.summary()

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 6s 230us/sample - loss: 0.6223 - accuracy: 0.6734
Epoch 2/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5680 - accuracy: 0.7219
Epoch 3/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5628 - accuracy: 0.7230s - loss: 0.5627 - accu
Epoch 4/100
25724/25724 [==============================] - 2s 92us/sample - loss: 0.5606 - accuracy: 0.7252
Epoch 5/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5591 - accuracy: 0.7259
Epoch 6/100
25724/25724 [==============================] - 2s 93us/sample - loss: 0.5579 - accuracy: 0.7269
Epoch 7/100
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5570 - accuracy: 0.7271
Epoch 8/100
25724/25724 [==============================] - 2s 93us/sample - loss: 0.5564 - accuracy: 0.7278
Epoch 9/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.555

25724/25724 [==============================] - 3s 111us/sample - loss: 0.5496 - accuracy: 0.7332
Epoch 74/100
25724/25724 [==============================] - 3s 121us/sample - loss: 0.5495 - accuracy: 0.7331
Epoch 75/100
25724/25724 [==============================] - 3s 112us/sample - loss: 0.5492 - accuracy: 0.7336
Epoch 76/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5493 - accuracy: 0.7336
Epoch 77/100
25724/25724 [==============================] - 3s 115us/sample - loss: 0.5493 - accuracy: 0.7327
Epoch 78/100
25724/25724 [==============================] - 3s 112us/sample - loss: 0.5494 - accuracy: 0.7339
Epoch 79/100
25724/25724 [==============================] - 3s 123us/sample - loss: 0.5493 - accuracy: 0.7334
Epoch 80/100
25724/25724 [==============================] - 5s 184us/sample - loss: 0.5491 - accuracy: 0.7339
Epoch 81/100
25724/25724 [==============================] - 3s 124us/sample - loss: 0.5493 - accuracy: 0.7331
Epoch 82/100
25724/2572

In [68]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test.values, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5505 - accuracy: 0.7237
Loss: 0.5583582179052836, Accuracy: 0.7237317562103271
